In [1]:
from datetime import datetime

def price_contract_v3(in_dates, in_prices, out_dates, out_prices, rate, storage_cost_rate, total_vol, injection_withdrawal_cost):
    """
    Simplified Gas Storage Pricing Model

    Parameters:
    - in_dates (list): List of injection dates (format: 'YYYY-MM-DD')
    - in_prices (list): Corresponding prices for injection dates
    - out_dates (list): List of withdrawal dates (format: 'YYYY-MM-DD')
    - out_prices (list): Corresponding prices for withdrawal dates
    - rate (float): Maximum volume that can be injected/withdrawn per operation (MMBtu)
    - storage_cost_rate (float): Monthly storage cost ($)
    - total_vol (float): Maximum storage capacity (MMBtu)
    - injection_withdrawal_cost (float): Cost per MMBtu for injection and withdrawal ($)

    Returns:
    - net_profit (float): The total net profit/loss from the contract ($)
    """
    # Convert dates to datetime
    in_dates = [datetime.strptime(date, "%Y-%m-%d") for date in in_dates]
    out_dates = [datetime.strptime(date, "%Y-%m-%d") for date in out_dates]
    all_dates = sorted(set(in_dates + out_dates))

    volume = 0  # Current storage volume
    buy_cost = 0  # Total purchase cost
    sell_revenue = 0  # Total revenue from sales
    storage_cost = 0  # Total storage cost

    for date in all_dates:
        if date in in_dates:
            idx = in_dates.index(date)
            price = in_prices[idx]

            # Ensure storage does not exceed maximum capacity
            if volume + rate > total_vol:
                print(f"⚠️ Skipped injection on {date.strftime('%Y-%m-%d')} (Insufficient storage capacity)")
                continue

            volume += rate
            buy_cost += price * rate
            storage_cost += storage_cost_rate  # Add monthly storage cost
            print(f"✅ Injected {rate} MMBtu on {date.strftime('%Y-%m-%d')} at ${price}/MMBtu")

        if date in out_dates:
            idx = out_dates.index(date)
            price = out_prices[idx]

            # Ensure there is enough gas to withdraw
            if volume < rate:
                print(f"⚠️ Skipped withdrawal on {date.strftime('%Y-%m-%d')} (Insufficient stored gas)")
                continue

            volume -= rate
            sell_revenue += price * rate
            print(f"✅ Withdrew {rate} MMBtu on {date.strftime('%Y-%m-%d')} at ${price}/MMBtu")

    # Calculate total injection & withdrawal costs
    transaction_cost = injection_withdrawal_cost * (len(in_dates) + len(out_dates)) * rate

    # Compute final contract value
    net_profit = sell_revenue - buy_cost - storage_cost - transaction_cost

    return round(net_profit, 2)


In [2]:
# Example Parameters
injection_dates = ["2024-06-30", "2024-07-31"]
withdrawal_dates = ["2024-09-30", "2024-12-31"]
injection_prices = [11.5, 11.7]  # Prices at injection dates ($/MMBtu)
withdrawal_prices = [12.3, 12.8]  # Prices at withdrawal dates ($/MMBtu)
rate = 500000  # MMBtu per injection/withdrawal
storage_cost_rate = 100000  # $ per month
total_vol = 5000000  # Maximum storage capacity (MMBtu)
injection_withdrawal_cost = 0.01  # $ per MMBtu for injection/withdrawal

# Run the simplified function
contract_value_v3 = price_contract_v3(
    injection_dates, injection_prices, withdrawal_dates, withdrawal_prices,
    rate, storage_cost_rate, total_vol, injection_withdrawal_cost
)

# Display the contract value
contract_value_v3

✅ Injected 500000 MMBtu on 2024-06-30 at $11.5/MMBtu
✅ Injected 500000 MMBtu on 2024-07-31 at $11.7/MMBtu
✅ Withdrew 500000 MMBtu on 2024-09-30 at $12.3/MMBtu
✅ Withdrew 500000 MMBtu on 2024-12-31 at $12.8/MMBtu


730000.0